# Eric Cestari
# TCC - Data Science & Analytics
# Predição de churn em uma empresa de telecomunicações através de modelos de Machine Learning

Dados https://community.ibm.com/accelerators/catalog/content/Telco-customer-churn

Kaggle https://www.kaggle.com/code/enanne/telco-churn-analysis-and-modeling/notebook

https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

Entendimento do estudo

Esse trabalho objetiva explorar os dados relacionados ao churn rate de clientes de uma empresa e entender como estão dispostos, descrevendo quais as possíveis variáveis que podem ou não influenciar a fidelização ou não dos clientes, trabalhar os dados e prepara-los para então desenvolver quatro modelos diferentes de machine learning, definindo qual modelo apresenta a maior acuracidade de predição, se o cliente tem ou não risco de churn, para servir como tomada de decisão para gestores.

1. Importando os dados
2. Limpeza e pré-processamento dos dados
3. Análise exploratória dos dados
4. Preparação dos dados para modelos de machine learning
5. Divisão dos dados em treino e teste
6. Treinamento dos dados nos modelos de Machine Learning
7. Avaliação de performance dos modelos

## Importando as bibliotecas

In [ ]:
# Manipulação dos dados
import pandas as pd
# Para trabalhar com matrizes
import numpy as np
# Gráficos
import seaborn as sns
# Interface gráfica
import matplotlib.pyplot as plt

import mglearn

# Pacotes para preparação dos dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import StandardScaler

# criação de modelos
# import statsmodel as sm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import KFold, cross_val_score

# importância de features
from sklearn.feature_selection import RFE

# validação
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_curve, auc
import graphviz

# lazy predict
import lazypredict
from lazypredict.Supervised import LazyClassifier

# Filtro de Warning
from warnings import simplefilter

# Ignorar warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
"""from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Importe os dados
X = [[0, 0], [1, 1], [2, 2], [3, 3]]
y = [0, 1, 2, 3]

# Crie o modelo de regressão linear
lr = LinearRegression()

# Crie o objeto RFE e ajuste o modelo
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X, y)

# Imprima o ranking das features
print("Ranking das Features:", rfe.ranking_)"""

# 1. Importando os dados

In [ ]:
dados = pd.read_excel('data_churn.xlsx')

## Entendendo os dados

In [ ]:
# Verificando a dimensão do DataFrame
print('O DataFrame possui {} observações (linhas) e {} variáveis (colunas).\n'.format(dados.shape[0], dados.shape[1]))

In [ ]:
dados.info()

In [ ]:
# Visualizar as 5 primeiras linhas
dados.head(5)

In [ ]:
# Traduzindo o nome das variáveis originais do Inglês para o Português
dados = dados.rename(columns={'CustomerID': 'IDUsuario', 'Count': 'Contagem', 'Country': 'Pais', 'State': 'Estado', 'City': 'Cidade',
                              'Zip Code': 'CodigoPostal', 'Lat Long': 'LatLong', 'Latitude': 'Latitude', 'Longitude': 'Longitude',
                              'Gender': 'Genero', 'Senior Citizen': 'Senior', 'Partner': 'Parceiro', 'Dependents': 'Dependentes',
                              'Tenure Months': 'AssinaturaMeses', 'Phone Service': 'ServicoTelefonia', 'Multiple Lines': 'MultiplasLinhas',
                              'Internet Service': 'ServicoInternet', 'Online Security': 'SegurancaOnline', 'Online Backup': 'BackupOnline',
                              'Device Protection': 'ProtecaoDispositivo', 'Tech Support': 'SuporteTech', 'Streaming TV': 'StreamingTV',
                              'Streaming Movies': 'StreamingFilmes', 'Contract': 'TipoContrato', 'Paperless Billing': 'FaturaSemPapel',
                              'Payment Method': 'MetodoPagamento', 'Monthly Charges': 'CobrancasMensais', 'Total Charges': 'CobrancasTotais',
                              'Churn Label': 'Churn', 'Churn Value': 'ValorChurn', 'Churn Score': 'PontuacaoChurn', 'CLTV': 'CLTV',
                              'Churn Reason': 'MotivoChurn'})

In [ ]:
dados.dtypes.value_counts()

In [ ]:
dados.describe().round(2)

Inicialmente, observei que existem poucas variáveis quantitativas no conjunto de dados

# 2. Limpeza e pré-processamento dos dados

In [ ]:
print('Dados faltantes \n{}.'.format(dados.isnull().sum() / len(dados)))

A variável 'CustomerID' não será importante para o modelo preditivo, uma vez que trata de um número único para cada cliente. Será deletada da base.
Também foi identificado através do resumo que as variáveis 'Count', 'Country' e 'State' possuem apenas um valor único e não são relevantes para a análise. Portanto, irei excluí-las.
Assumimos que o dataset traz informações de clientes dos Estados Unidos, especificamente da Calofornia.

In [ ]:
# As variáveis 'CustomerID', 'Count', 'Country', 'State' também não serão utilizadas as variáveis 'Zip Code', 'Lat Long', 'Latitude' e 'Longitude' não agregam valor a análise, portanto serão descartadas
dados.drop(['IDUsuario', 'Contagem', 'Pais', 'Estado', 'CodigoPostal', 'LatLong', 'Latitude', 'Longitude'], inplace=True, axis=1)

In [ ]:
# Também será descartada a variável 'Churn Label', pois já temos o mapeamento de churn feito na variável 'Churn Value' sendo 1 para churn e 0 para não churn
#dados.drop(['Churn Label'], inplace=True, axis=1)

In [ ]:
# Para mais detalhes do conjunto de dados
dados.info()

A variável 'CobrancasTotais' está em formato 'object', será necessário transforma-la no tipo 'float' para dar andamento a analise

In [ ]:
# Ao tentar transformar a variável 'Total Charges' que atualmente é um object em float, o Python retorna um erro
dados['CobrancasTotais'].astype(float)

In [ ]:
# Para transformar o valor vazio (" ") em string para depois transformar em float, irei utilizar a seguinte função
def string_para_float(entrada):
    try:
        return float(entrada)
    except ValueError:
        return np.nan

In [ ]:
# Aplicando a função
dados['CobrancasTotais'] = dados['CobrancasTotais'].apply(string_para_float)

In [ ]:
print('Dados faltantes em "CobrancasTotais" anteriormente: {}'.format(dados['CobrancasTotais'].isnull().sum()))

In [ ]:
# Preencher dados faltantes com a média da variável
Media_CobrancasTotais = dados['CobrancasTotais'].mean()
dados['CobrancasTotais'].fillna(Media_CobrancasTotais, inplace=True)

In [ ]:
print('Dados faltantes em "CobrancasTotais" após a função: {}'.format(dados['CobrancasTotais'].isnull().sum()))

In [ ]:
dados.isnull().sum()

In [ ]:
dados.isna().sum()

In [ ]:
# Função para identificar quais valores únicos e suas quantidades em cada variável

def resumo(dados):
    col = []
    d_type = []
    uniques = []
    n_uniques = []
    
    for i in dados.columns:
        col.append(i)
        d_type.append(dados[i].dtypes)
        uniques.append(dados[i].unique()[:5])
        n_uniques.append(dados[i].nunique())
    
    return pd.DataFrame({'variavel': col, 'd_type': d_type, 'unico': uniques, 'quantidade': n_uniques})

resumo(dados)

In [ ]:
# Com o objetivo de padronizar as variáveis, substitui "No phone service" por apenas "No"
variaveis1 = ['MultiplasLinhas']

for i in variaveis1:
    dados[i] = dados[i].replace({"No phone service" : "No"})

In [ ]:
# Também com o objetivo de padronizar as variáveis, substitui "No internet service" por apenas "No"
variaveis2 = ['SegurancaOnline', 'BackupOnline', 'ProtecaoDispositivo', 'SuporteTech', 'StreamingTV', 'StreamingFilmes']

for i in variaveis2:
    dados[i] = dados[i].replace({"No internet service" : "No"})

In [ ]:
resumo(dados)

# 3. Análise exploratória dos dados

In [ ]:
# Analisando apenas as variáveis categoricas
dados.select_dtypes(include='object').describe()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(dados['Churn'], order=dados['Churn'].value_counts().index[::1])
plt.title('Distribuição de Churn')
plt.xlabel('Churn')
plt.ylabel('Quantidade de clientes')
plt.show()

In [ ]:
# Churn
print('0 não churn e 1 churn\n',(dados['Churn'].value_counts() / dados.shape[0] * 100).round(2))

In [ ]:
# Analisando o churn por Cidade
"""contract_churn = dados.groupby(['Cidade', 'Churn']).size().unstack()
contract_churn.plot(kind='bar', stacked=True)
plt.xlabel('Cidade')
plt.ylabel('Número de Clientes')
plt.title('Churn por Cidade')
plt.show()"""

In [ ]:
# gráfico de barras do churn por Gênero
gender_churn = dados.groupby(['Genero', 'Churn'])['Churn'].count().unstack()
gender_churn.plot(kind='bar', stacked=True)
plt.title('Churn por gênero')
plt.xlabel('Gênero')
plt.ylabel('Contagem')
plt.show()

In [ ]:
frequencia_genero = dados['Genero'].value_counts()
percentual_genero = dados['Genero'].value_counts(normalize = True)*100
percentual_genero = pd.DataFrame({'Frequência': frequencia_genero, 'Porcentagem(%)': percentual_genero})
percentual_genero

In [ ]:
# gráfico de barras do churn por Senior
gender_churn = dados.groupby(['Senior', 'Churn'])['Churn'].count().unstack()
gender_churn.plot(kind='bar', stacked=True)
plt.title('Churn por Senior')
plt.xlabel('Senior')
plt.ylabel('Contagem')
plt.show()

In [ ]:
frequencia_senior = dados['Senior'].value_counts()
percentual_senior = dados['Senior'].value_counts(normalize = True)*100
percentual_senior = pd.DataFrame({'Frequência': frequencia_senior, 'Porcentagem(%)': percentual_senior})
percentual_senior

In [ ]:
# gráfico de barras do churn por Parceiro
gender_churn = dados.groupby(['Parceiro', 'Churn'])['Churn'].count().unstack()
gender_churn.plot(kind='bar', stacked=True)
plt.title('Churn por Parceiro')
plt.xlabel('Parceiro')
plt.ylabel('Contagem')
plt.show()

In [ ]:
frequencia_parceiro = dados['Parceiro'].value_counts()
percentual_parceiro = dados['Parceiro'].value_counts(normalize = True)*100
percentual_parceiro = pd.DataFrame({'Frequência': frequencia_parceiro, 'Porcentagem(%)': percentual_parceiro})
percentual_parceiro

In [ ]:
# gráfico de barras do churn por Dependentes
gender_churn = dados.groupby(['Dependentes', 'Churn'])['Churn'].count().unstack()
gender_churn.plot(kind='bar', stacked=True)
plt.title('Churn por Dependentes')
plt.xlabel('Dependentes')
plt.ylabel('Contagem')
plt.show()

In [ ]:
frequencia_dependentes = dados['Dependentes'].value_counts()
percentual_dependentes = dados['Dependentes'].value_counts(normalize = True)*100
percentual_dependentes = pd.DataFrame({'Frequência': frequencia_dependentes, 'Porcentagem(%)': percentual_dependentes})
percentual_dependentes

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(dados['AssinaturaMeses'], bins=30, kde=True)
plt.title('Distribuição de AssinaturaMeses')
plt.xlabel('AssinaturaMeses')
plt.ylabel('Contagem')
plt.show()

In [ ]:
f, ax = plt.subplots(3,3, figsize=(20,15))
sns.countplot(dados['ServicoTelefonia'], ax=ax[0,0])
sns.countplot(dados['MultiplasLinhas'], ax=ax[0,1])
sns.countplot(dados['ServicoInternet'], ax=ax[0,2])
sns.countplot(dados['SegurancaOnline'], ax=ax[1,0])
sns.countplot(dados['BackupOnline'], ax=ax[1,1])
sns.countplot(dados['ProtecaoDispositivo'], ax=ax[1,2])
sns.countplot(dados['SuporteTech'], ax=ax[2,0])
sns.countplot(dados['StreamingTV'], ax=ax[2,1])
sns.countplot(dados['StreamingFilmes'], ax=ax[2,2])

In [ ]:
# Analisando o churn por categoria de Contrato
contract_churn = dados.groupby(['TipoContrato', 'Churn']).size().unstack()
contract_churn.plot(kind='bar', stacked=True)
plt.xlabel('Tipo de Contrato')
plt.ylabel('Número de Clientes')
plt.title('Churn por Tipo de Contrato')
plt.show()

In [ ]:
frequencia_faturasempapel = dados['FaturaSemPapel'].value_counts()
percentual_faturasempapel = dados['FaturaSemPapel'].value_counts(normalize = True)*100
percentual_faturasempapel = pd.DataFrame({'Frequência': frequencia_faturasempapel, 'Porcentagem(%)': percentual_faturasempapel})
percentual_faturasempapel

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, sharey = True, figsize = (20,6))

ax = sns.distplot(dados[dados['TipoContrato']=='Month-to-month']['AssinaturaMeses'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'blue',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax1)
ax.set_ylabel('Quantidade de Clientes')
ax.set_xlabel('Assinatura (meses)')
ax.set_title('Contrato mês a mês')

ax = sns.distplot(dados[dados['TipoContrato']=='One year']['AssinaturaMeses'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'orange',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax2)
ax.set_xlabel('Assinatura (meses)',size = 14)
ax.set_title('Contrato de um ano',size = 14)

ax = sns.distplot(dados[dados['TipoContrato']=='Two year']['AssinaturaMeses'],
                   hist=True, kde=False,
                   bins=int(180/5), color = 'green',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax3)

ax.set_xlabel('Assinatura (meses)')
ax.set_title('Contrato de dois anos')

In [ ]:
# Analisando o churn por Metodo de Pagamento
contract_churn = dados.groupby(['MetodoPagamento', 'Churn']).size().unstack()
contract_churn.plot(kind='bar', stacked=True)
plt.xlabel('Metodo de Pagamento')
plt.ylabel('Número de Clientes')
plt.title('Churn por Metodo Pagamento')
plt.show()

In [ ]:
frequencia_metodopagamento = dados['MetodoPagamento'].value_counts()
percentual_metodopagamento = dados['MetodoPagamento'].value_counts(normalize = True)*100
percentual_metodopagamento = pd.DataFrame({'Frequência': frequencia_metodopagamento, 'Porcentagem(%)': percentual_metodopagamento})
percentual_metodopagamento

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(dados['CobrancasMensais'], bins=30, kde=True)
plt.title('Cobranças Mensais')
plt.xlabel('Valor Cobranças Mensais')
plt.ylabel('Contagem')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(dados['CobrancasTotais'], bins=30, kde=True)
plt.title('Cobranças Totais')
plt.xlabel('Valor')
plt.ylabel('Contagem')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(dados['PontuacaoChurn'], bins=30, kde=True)
plt.title('PontuacaoChurn')
plt.xlabel('Pontuação')
plt.ylabel('Contagem')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(dados['CLTV'], bins=30, kde=True)
plt.title('CLTV')
plt.xlabel('Valor')
plt.ylabel('Contagem')
plt.show()

In [ ]:
#dadoscorr = dados.drop('ValorChurn', axis=1, inplace=True)

In [ ]:
# criar a matriz de correlação
corr_matrix = dados.corr()

In [ ]:
# criar o heatmap
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm',
            fmt='.2f', linewidths=.05, annot_kws={"size": 12})
plt.title('Correlação das variáveis com Churn')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
dados.select_dtypes(include='int64').describe().round(2)

In [ ]:
dados.select_dtypes(include='float64').describe().round(2)

In [ ]:
sns.boxplot(dados['AssinaturaMeses'])

In [ ]:
sns.boxplot(dados['PontuacaoChurn'])

In [ ]:
sns.boxplot(dados['CLTV'])

In [ ]:
sns.boxplot(dados['CobrancasMensais'])

Concluo que não existem dados outliers analisando o gráfico boxplot para a variável 'Monthly Charges'.

Também concluo que não existem dados outliers analisando o gráfico boxplot para a variável 'Total Charges'.

In [ ]:
sns.boxplot(dados['CobrancasTotais'])

In [ ]:
sns.boxplot('Churn', 'CobrancasMensais', data=dados)

In [ ]:
# Agrupar os dados por Churn e calcular estatísticas descritivas para cada grupo
stats = dados.groupby('Churn')['CobrancasMensais'].describe()

# Imprimir tabela com as estatísticas
print(stats)

O valor mediano é superior nos clientes em Churn em relação aos Não Churn

In [ ]:
# Criando uma nova variável para 'AssinaturaMeses' agrupando em classes, para melhor entendimento
# 0 a 12 meses
# 13 a 24 meses
# 24 a 48 meses
# 48 a 60 meses
# 60 + meses
classes = [0, 12, 24, 48, 60, 100]

tenure = pd.cut(dados['AssinaturaMeses'], classes)

pd.value_counts(tenure)

In [ ]:
labels = ['0 a 12 meses', '13 a 24 meses', '24 a 48 meses', '48 a 60 meses', '60 meses +']

In [ ]:
tenure = pd.cut(dados['AssinaturaMeses'], classes, labels=labels)
pd.value_counts(tenure)

In [ ]:
tenure = pd.cut(dados['AssinaturaMeses'], classes, labels=labels, include_lowest=True)
pd.value_counts(tenure)

In [ ]:
# Faixas temporais de permanência no serviço
dados['AssinaturaMeses'].value_counts().plot(kind='bar')

In [ ]:
# Justificativas dos clientes para sair do serviço
dados['MotivoChurn'].value_counts().plot(kind='bar')

In [ ]:
motivo_churn = (dados['MotivoChurn'].value_counts() / dados['MotivoChurn'].value_counts().sum() * 100).round(2)
motivo_churn

In [ ]:
print('Justificativa \n',dados['MotivoChurn'].value_counts() / dados.shape[0] * 100)

# 4. Preparação dos dados para modelos de machine learning

In [ ]:
dados.drop(['Churn'], inplace=True, axis=1)

As variáveis categóricas devem ser transformadas em variáveis "dummies" para a aplicação no modelo de machine learning

In [ ]:
dados_dummies = pd.get_dummies(dados, columns=['Cidade'], prefix='Cidade')
dados_dummies = pd.get_dummies(dados_dummies, columns=['Genero'], prefix='Genero')
dados_dummies = pd.get_dummies(dados_dummies, columns=['Senior'], prefix='Senior')
dados_dummies = pd.get_dummies(dados_dummies, columns=['Parceiro'], prefix='Parceiro')
dados_dummies = pd.get_dummies(dados_dummies, columns=['Dependentes'], prefix='Dependentes')
dados_dummies = pd.get_dummies(dados_dummies, columns=['ServicoTelefonia'], prefix='ServicoTelefonia')
dados_dummies = pd.get_dummies(dados_dummies, columns=['MultiplasLinhas'], prefix='MultiplasLinhas')
dados_dummies = pd.get_dummies(dados_dummies, columns=['ServicoInternet'], prefix='ServicoInternet')
dados_dummies = pd.get_dummies(dados_dummies, columns=['SegurancaOnline'], prefix='SegurancaOnline')
dados_dummies = pd.get_dummies(dados_dummies, columns=['BackupOnline'], prefix='BackupOnline')
dados_dummies = pd.get_dummies(dados_dummies, columns=['ProtecaoDispositivo'], prefix='ProtecaoDispositivo')
dados_dummies = pd.get_dummies(dados_dummies, columns=['SuporteTech'], prefix='SuporteTech')
dados_dummies = pd.get_dummies(dados_dummies, columns=['StreamingTV'], prefix='StreamingTV')
dados_dummies = pd.get_dummies(dados_dummies, columns=['StreamingFilmes'], prefix='StreamingFilmes')
dados_dummies = pd.get_dummies(dados_dummies, columns=['TipoContrato'], prefix='TipoContrato')
dados_dummies = pd.get_dummies(dados_dummies, columns=['FaturaSemPapel'], prefix='FaturaSemPapel')
dados_dummies = pd.get_dummies(dados_dummies, columns=['MetodoPagamento'], prefix='MetodoPagamento')
dados_dummies = pd.get_dummies(dados_dummies, columns=['MotivoChurn'], prefix='MotivoChurn')


dados_dummies.head()

In [ ]:
# Ajustando o nome de cada variável, removendo espaços e adicionando 'underscore' no lugar
dados_dummies.columns = [i.replace(' ', '_') for i in dados_dummies.columns]
dados_dummies

# 5. Divisão dos dados em treino e teste

## Árvore de Decisão

In [ ]:
X = dados_dummies.drop('ValorChurn', axis=1)
y = dados_dummies['ValorChurn']

In [ ]:
labels = y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=1)

# 6. Treinamento dos dados nos modelos de Machine Learning

## Modelo 1 - Sem ajustes

In [ ]:
modelo1 = DecisionTreeClassifier(max_depth=7,
                                 max_features=None,
                                 criterion='entropy',
                                 min_samples_leaf=1,
                                 min_samples_split=2,
                                 random_state=1)

In [ ]:
modelo1.fit(X_train, y_train)

In [ ]:
print("Accuracy on training set: {:.3f}".format(modelo1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(modelo1.score(X_test, y_test)))

In [ ]:
dot_data = export_graphviz(modelo1, out_file=None,
                           class_names=["não churn", "churn"],
                           feature_names= X.columns.tolist(),
                           impurity=False, 
                           filled=True, rounded=True,
                           special_characters=True)

In [ ]:
graphviz.Source(dot_data)

In [ ]:
export_graphviz(modelo1, out_file="modelo1.dot", class_names=["nao churn", "churn"],
               impurity=False, filled=True)

In [ ]:
import graphviz
with open("modelo1.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

## Modelo 2 - Com ajustes

In [ ]:
modelo2 = DecisionTreeClassifier(max_depth=4,
                                 max_features=None,
                                 criterion='entropy',
                                 min_samples_leaf=1,
                                 min_samples_split=2,
                                 random_state=1)

In [ ]:
modelo2.fit(X_train, y_train)

In [ ]:
print("Accuracy on training set: {:.4f}".format(modelo2.score(X_train, y_train)))
print("Accuracy on test set: {:.4f}".format(modelo2.score(X_test, y_test)))

In [ ]:
export_graphviz(modelo2, out_file="modelo2.dot", class_names=["não churn", "churn"],
               impurity=False, filled=True)

In [ ]:
import graphviz
with open("modelo2.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

In [ ]:
print("Feature importances:\n{}".format(modelo1.feature_importances_))

## Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=5, random_state=2)
y_pred_rf = rf.fit(X_train, y_train)

In [ ]:
print("Acuracidade nos dados de treino: {:.4f}".format(rf.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(rf.score(X_test, y_test)))

In [ ]:
# Definir o método de validação cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')
print('Acurácia média:', scores.mean())

# XGBoost

In [ ]:
xgb = XGBClassifier(random_state=2)
y_pred_xgb = xgb.fit(X_train, y_train)

In [ ]:
print("Acuracidade nos dados de treino: {:.4f}".format(xgb.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(xgb.score(X_test, y_test)))

## Regressão Logística

In [ ]:
rl = LogisticRegression(random_state=2)
y_pred_rl = rl.fit(X_train, y_train)

In [ ]:
rfe = RFE(rl, n_features_to_select=1)
rfe.fit(X, y)

In [ ]:
# Calcular a probabilidade de classificação no conjunto de teste
probabilidades = rl.predict_proba(X_test)[:, 1]

In [ ]:
# Calcular a curva ROC e a área sob a curva (AUC)
fpr, tpr, limiares = roc_curve(y_test, probabilidades)
roc_auc = auc(fpr, tpr)

In [ ]:
print("Acuracidade nos dados de treino: {:.4f}".format(rl.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(rl.score(X_test, y_test)))

## Lazy Predict

In [ ]:
# lazy predict
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models_train,predictions_train = clf.fit(X_train, X_train, y_train, y_train)
models_test,predictions_test = clf.fit(X_train, X_test, y_train, y_test)

# performance dos modelos
models_train

# 7. Avaliação de performance dos modelos

In [ ]:
#ideias para avaliação
#acuracia
#f1
#recall
#auc roc
#confusion matrix
# MAE
# Cross Validation

In [ ]:
#Árvore de Decisão - Sem ajuste
print("Accuracy on training set: {:.3f}".format(modelo1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(modelo1.score(X_test, y_test)))

In [ ]:
#Árvore de Decisão - Com ajuste
print("Accuracy on training set: {:.4f}".format(modelo2.score(X_train, y_train)))
print("Accuracy on test set: {:.4f}".format(modelo2.score(X_test, y_test)))

In [ ]:
#Random Forest
print("Acuracidade nos dados de treino: {:.4f}".format(rf.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(rf.score(X_test, y_test)))

In [ ]:
#XGBoost
print("Acuracidade nos dados de treino: {:.4f}".format(xgb.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(xgb.score(X_test, y_test)))

In [ ]:
#Regressão Logística
print("Acuracidade nos dados de treino: {:.4f}".format(rl.score(X_train, y_train)))
print("Acuracidade nos dados de teste: {:.4f}".format(rl.score(X_test, y_test)))

In [ ]:
# Analisando o churn por categoria de contrato
contract_churn = telco_data.groupby(['Contract', 'Churn']).size().unstack()
contract_churn.plot(kind='bar', stacked=True)
plt.xlabel('Tipo de Contrato')
plt.ylabel('Número de Clientes')
plt.title('Churn por Tipo de Contrato')
plt.show()